In [56]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [57]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,35881556553,18397231064,310731060,310731060,268358393,268358393,197834085,197834085,385305779,183233839
2,34434707268,18820831904,123100573,118688191,98572943,98572943,86678231,86678231,372596872,239177970
3,70393547201,17662294326,548025869,522263272,457143583,457143583,334637608,334637608,681622354,375623182
4,62648026006,18790887173,147443942,147443942,110160319,110160319,85408682,85408682,544622354,458477529


In [58]:
# Parametes 
TRAIN_SPLIT = 30000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 16 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 30000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 1875 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 750 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 10
bs_capacity = 20000000000


In [59]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [60]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [61]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [62]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 1875 steps, validate for 750 steps
Epoch 1/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0124 - val_loss: 0.0085
Epoch 2/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0070 - val_loss: 0.0068
Epoch 3/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0065 - val_loss: 0.0076
Epoch 4/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0061 - val_loss: 0.0076
Epoch 5/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0059 - val_loss: 0.0072
Epoch 6/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0058 - val_loss: 0.0079
Epoch 7/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0057 - val_loss: 0.0071
Epoch 8/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0056 - val_loss: 0.0079
Epoch 9/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0055 - val_loss: 0.0077
Epoch 10/10
1875/18

In [64]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[35881556553 , 310731060 , 268358393, 197834085, 385305779]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[0]) # because I took the input of the first values in x_train for test


[0.9523152  0.00701474 0.00612248 0.00920417 0.00857242]
[0.93077304 0.01325895 0.01335097 0.00897079 0.01115676]
[0.94691247 0.00723898 0.00671243 0.00945676 0.00934454]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ]


In [66]:
# Save trained model
model.save("model.h5")


In [ ]:
[0.9490212  0.00932256 0.00688147 0.00760134 0.01129607]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ] 42 256 32
[0.9396471  0.00828853 0.0097623  0.0081543  0.01002839]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ]